#Image Classification with Pytorch a pretrained Resnet-50 model

In [ ]:
# If needed:
# !pip -q install gradio torch torchvision pillow requests

import torch
from PIL import Image
import gradio as gr
from torchvision.models import resnet50, ResNet50_Weights

# -----------------------------
# 1) Model + labels (ResNet-50)
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights).to(device).eval()

# Human-readable ImageNet labels bundled with weights
labels = weights.meta["categories"]

# -----------------------------
# 2) Preprocessing (ImageNet standard for these weights)
# -----------------------------
preprocess = weights.transforms()

# -----------------------------
# 3) Prediction function
# -----------------------------
@torch.no_grad()
def predict(image: Image.Image):
    """
    Input: PIL image
    Output: dict[label -> probability] for Gradio
    """
    if image is None:
        return {}

    image = image.convert("RGB")
    x = preprocess(image).unsqueeze(0).to(device)  # [1, 3, 224, 224]

    logits = model(x)[0]
    probs = torch.softmax(logits, dim=0)

    # Gradio Label expects a dict of confidences
    return {labels[i]: float(probs[i].cpu()) for i in range(len(labels))}

# -----------------------------
# 4) Example images (URLs)
# -----------------------------
example_1 = "https://images.unsplash.com/photo-1503376780353-7e6692767b70"  # sports car
example_2 = "https://images.unsplash.com/photo-1436491865332-7a61a109cc05"  # airplane

# -----------------------------
# 5) Gradio UI
# -----------------------------
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Input Image"),
    outputs=gr.Label(num_top_classes=3, label="Top Predictions"),
    examples=[[example_1], [example_2]],
    title="Image Classification (ResNet-50, ImageNet)",
    description="Upload an image (or pick an example) to classify it with a pretrained ResNet-50 model.",
)

demo.launch()
